In [123]:
import numpy as np
import pandas as pd
import pickle as pickle
import codecs
import requests
import bs4
import json
import warnings
import re
import os

from sklearn.utils import shuffle
from tqdm import tqdm
from multiprocessing import Pool
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

Из-за нетипичного для csv-файла формата (по факту это html или же xml) прочитать тестовые данные с помощью pandas.read_csv не получится. Поэтому воспользуемся библиотекой BeautifulSoup

In [124]:
with codecs.open('test.csv', 'r') as f:
    test_text = f.read()
    soup = bs4.BeautifulSoup(test_text, 'html.parser')
    test_reviews = soup.findAll('review')
    test_list = [test_review.text for test_review in test_reviews]

In [125]:
test = pd.DataFrame(test_list, columns=['text'])

In [126]:
test.head()

,text
0,"Ужасно слабый аккумулятор, это основной минус ..."
1,ценанадежность-неубиваемостьдолго держит батар...
2,"подробнее в комментариях\nК сожалению, факт по..."
3,я любительница громкой музыки. Тише телефона у...
4,"Дата выпуска - 2011 г, емкость - 1430 mAh, тех..."


Данные для обучающей выборки будем брать на площадке Яндекс.Маркет. Тестовая выборка содержит отзывы на смартфоны, поэтому определиться с необходимым разделом сайта нетрудно.

In [127]:
def get_mobiles():   
    mobile_list = []  
    for i in range(1, 11): 
        url = "https://market.yandex.ru/catalog--mobilnye-telefony/54726/list"
        params = {
            'onstock': 1,
            'local-offers-first': 0,
            'how': "opinions",
            'page': i
        }
        html = requests.get(url, params, headers={'User-Agent': 'Mozilla/5.0'}).text
        if html:
            soup = bs4.BeautifulSoup(html, 'html.parser')     
            mobiles = soup.findAll('div', class_='n-snippet-cell2 i-bem b-zone b-spy-visible n-snippet-cell2_type_product')
            for mobile in mobiles:
                mobile_link = mobile.find('a', class_='n-snippet-cell2__image link').get('href')
                mobile_url = mobile_link.split('?')
                if mobile_url[0] not in mobile_list:
                    mobile_name = mobile_url[0].split('/')                    
                    mobile_list.append({
                        'url': mobile_url[0],
                        'name': mobile_name[1][18:].capitalize(),
                        'id': mobile_name[2]
                    }) 
    return mobile_list

In [128]:
def get_reviews(mobile):
    for i in range(1, 11):
        url = "https://market.yandex.ru{}/reviews".format(mobile['url'])
        params = {
            "hid": "91491",
            'page': i        
        }
        try:
            while True:
                html = requests.get(url, params, headers={'User-Agent': 'Mozilla/5.0'}).text
                if html:
                    soup = bs4.BeautifulSoup(html, 'html.parser')
                    reviews = soup.findAll('div', itemprop='review')
                    for review in reviews:
                        review_text = review.find('meta', itemprop='description').get('content')
                        review_rating = int(review.find('meta', itemprop='ratingValue').get('content'))
                        with open('data/train.json', 'a') as f:
                            json.dump({'text': review_text, 'rating': review_rating}, f, ensure_ascii=False)
                            f.write('\n')
                    if len(reviews) > 0:
                        break
                else:
                    print("Не удалось загрузить страницу")

        except:
            print("Сетевая ошибка")

In [129]:
mobiles = get_mobiles()
len(mobiles)

0

In [130]:
pool = Pool(10)
for _ in tqdm(pool.imap(get_reviews, mobiles), total=len(mobiles)):
    pass

0it [00:00, ?it/s]


In [131]:
def extract_info(text, label):
    try:
        if label == 1:
            text = re.findall(r'Достоинства(:.*?)Недостатки:', text)[0]
        else:
            text = re.findall(r'Недостатки(:.*?)Комментарий:', text)[0]
        return re.sub(r'\W', ' ', text)
    except:
        return text

In [132]:
train = pd.read_json('data/train.json', orient='records', lines=True).drop_duplicates()

In [133]:
train['label'] = train['rating'].apply(lambda x: int(x == 5))
train['length'] = train['text'].apply(lambda x: len(x))
train['text'] = train.apply(lambda x: extract_info(x['text'], x['label']), axis=1)

In [134]:
train.shape

(5000, 4)

In [135]:
train.head()

,rating,text,label,length
0,4,"Достоинства: 1) отличный дизайн, мой синий кор...",0,1294
1,5,"Достоинства: Качество связи,сборки.Удобство qw...",1,1321
2,5,Достоинства: Могу сказать сразу. Телефон шикар...,1,480
3,5,Достоинства: -качественная сборка -QWERTY-клав...,1,904
4,5,Достоинства: Купил вчера. во-первых это полная...,1,2639


In [136]:
train = shuffle(pd.concat([train[train['label'] == 0], 
                           train[train['label'] == 1].sort_values('length', ascending=False).head(1680)]), 
                           random_state=777)

In [137]:
X = train['text'].values
y = train['label'].values

In [138]:
X_test = test['text'].values

Будем использовать связку TfidfVectorizer и LinearSVC с подобранными опытным путём параметрами.

In [139]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.98, ngram_range=(1,2), sublinear_tf=True, analyzer='word', smooth_idf=True, norm='l2')
#создадим словарь грам на основе тестовых и тренировочных данных
vectorizer.fit(np.concatenate((np.array(X), np.array(X_test))))

TfidfVectorizer(analyzer='word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=0.98, max_features=None, min_df=10,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=True,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [140]:
from sklearn.svm import LinearSVC

In [141]:
svc = LinearSVC(penalty='l2', C=0.9, fit_intercept=False, dual=True, tol=0.0001, max_iter=200, loss='hinge')

In [142]:
from sklearn.model_selection import cross_val_score

In [150]:
cross_val_score(svc, vectorizer.transform(np.array(X)), y, cv=20).mean()

0.7657738095238095

In [144]:
svc.fit(vectorizer.transform(np.array(X)), y)
preds = svc.predict(vectorizer.transform(np.array(X_test)))

In [145]:
ans = pd.DataFrame({'Id' : np.arange(0, len(preds)), 'y' : np.array(['pos' if x == 1 else 'neg' for x in preds])})

In [146]:
ans.head()

,Id,y
0,0,neg
1,1,pos
2,2,neg
3,3,neg
4,4,pos


In [147]:
ans.to_csv('submission.csv', index=False)

In [148]:
output = open('model.pkl', 'wb')
pickle.dump(svc, output, 2)

In [151]:
output = open('vectorizer.pkl', 'wb')
pickle.dump(vectorizer, output, 2)